In [254]:
from os import walk

fitsfiles = [] 
for (dirpath, dirnames, filenames) in walk('./fitsfiles'):
    for file in filenames:    
        if(file.endswith('.fits')):
            if('pfss_intoout' in file):
                fitsfiles.append((dirpath + '/' + file, 'PFSS_IO'))
            elif('pfss_outtoin' in file):
                fitsfiles.append((dirpath + '/' + file, 'PFSS_OI'))
            elif('scs_outtoin' in file):
                fitsfiles.append((dirpath + '/' + file, 'SCS_OI'))
    break

### Choose indices

In [255]:
def everynth(step):
    return (range(0,16200, step), 'step' + str(step))

indices = everynth(25)

### Defintion from OpenSpace function FieldlinesState::saveStateToOsfls
    File is structured like this: (for version 0)
    0. int                    - version number of binary state file! (in case something needs to be altered in the future, then increase CurrentVersion)
    1. double                 - _triggerTime
    2. int                    - _model
    3. bool                   - _isMorphable
    4. size_t                 - Number of lines in the state  == _lineStart.size() == _lineCount.size()
    5. size_t                 - Total number of vertex points  == _vertexPositions.size() == _extraQuantities.size()
    6. size_t                 - Number of extra quantites  == _extraQuantities.size() == _extraQuantityNames.size()
    7. site_t                 - Number of total bytes that ALL _extraQuantityNames  consists of (Each such name is stored as a c_str which means it ends with the null char '\0' )
    8. std::vector<GLint>     - _lineStart
    9. std::vector<GLsizei>   - _lineCount
    10. std::vector<glm::vec3> - _vertexPositions
    11. std::vector<float>     - _extraQuantities
    12. array of c_str         - Strings naming the extra quantities (elements of _extraQuantityNames). Each string ends with null char '\0'

In [256]:
import math, numpy

def sph2cart(coord):
    return [
         coord[2] * math.sin(coord[1]) * math.cos(coord[0]), 
         coord[2] * math.sin(coord[1]) * math.sin(coord[0]),
         coord[2] * math.cos(coord[1])
    ]

def coord2datarow(coord):
    sph = sph2cart(coord[0:3])
    sph.append(abs(coord[5]))
    return [numpy.float32(x) for x in sph] 

In [257]:
from enum import Enum
class Model(Enum):
    Batsrus = 0
    Enlil = 1
    Pfss = 2
    Wsa = 3
    Invalid = 5

In [309]:
from astropy.io import fits
from astropy.time import Time

def toOsfls(filename, modelname, indices,  typename):
    fl_fits = fits.open(filename)
    fl_data = fl_fits[0].data
    fl_fits.close()
    versionNumber = 0
    [triggerTime, pathSafeTimeString] = obstimeToJ2000(fl_fits[0].header['OBSTIME'])
    print("TriggerTime: ",triggerTime)
    print("pathSafeTimeString: ",pathSafeTimeString)
    fileName = pathSafeTimeString + '.osfls'

    model = Model.Wsa.value
    isMorphable = False

    nVert = 0
    lineStart = []
    lineCount = []
    vertexPositions = []
    extraQuantities = []

    for i in indices:
        points = [coord2datarow(pt) for pt in fl_data[i] if pt[0] > -900] 
        if (len(points) < 2): continue
        lineStart.append(nVert)
        nVert += len(points)
        lineCount.append(len(points))
        [vertexPositions.extend(pt[0:3]) for pt in points] # extend to unfold elements
        [extraQuantities.append(pt[3]) for pt in points]

    nLines = len(lineStart)

    nExtras = 1
    extraQuantityNames = ['b-f mag\0']
    nStringBytes = sum([len(s) for s in extraQuantityNames])
    allNamesInOne = ''
    for s in extraQuantityNames:
        allNamesInOne += s
    
    # Prepare data for writing to binary. Using Struct and pack
    typestr = '= i d i ? Q Q Q Q %sl %sL %sf %sf %ss' % (nLines, nLines, 3*nVert, nVert, nStringBytes)
    struct_to_write = struct.Struct(typestr)
    #print('Format string  :', struct_to_write.format)
    print('Uses           :', struct_to_write.size, 'bytes')
    values_to_write = (versionNumber, triggerTime, model, isMorphable, nLines, nVert, nExtras, nStringBytes)
    values_to_write += (*lineStart, *lineCount, *vertexPositions, *extraQuantities, allNamesInOne.encode('utf-8'))
    
    buffer = ctypes.create_string_buffer(struct_to_write.size)    
    struct_to_write.pack_into(buffer, 0, *values_to_write)
    
    fout = open('./' + modelname + '/' + typename + '_' + fileName, 'wb')
    fout.write(buffer)
    fout.close()


### Make into osfls format

In [321]:
import time
start_time = time.time()
for fitsfile in fitsfiles:
    toOsfls(fitsfile[0], fitsfile[1], indices[0], indices[1])
    print('Finished converting {} after {} seconds: '.format(fitsfile[0],time.time()-start_time))
print("Execution time for type {}: {} seconds".format(indices[1], time.time()-start_time))


TriggerTime:  610056000.0000134
pathSafeTimeString:  2019-05-02T08-00-00.000
Uses           : 1659145 bytes
Finished converting ./fitsfiles/201905020800R000_trace_pfss_outtoin_tracing.fits after 0.6246249675750732 seconds: 
TriggerTime:  610056000.0000134
pathSafeTimeString:  2019-05-02T08-00-00.000
Uses           : 2186761 bytes
Finished converting ./fitsfiles/201905020800R000_trace_scs_outtoin_tracing.fits after 1.4091100692749023 seconds: 
TriggerTime:  610070400.0
pathSafeTimeString:  2019-05-02T12-00-00.000
Uses           : 2190089 bytes
Finished converting ./fitsfiles/201905021200R000_trace_scs_outtoin_tracing.fits after 2.7533059120178223 seconds: 
TriggerTime:  610084799.9999866
pathSafeTimeString:  2019-05-02T16-00-00.000
Uses           : 1657577 bytes
Finished converting ./fitsfiles/201905021600R000_trace_pfss_outtoin_tracing.fits after 3.766940116882324 seconds: 
TriggerTime:  610099200.0000134
pathSafeTimeString:  2019-05-02T20-00-00.000
Uses           : 1656777 bytes
Finis

In [320]:


def obstimeToJ2000(ot):
    timefits = ot[0:4] + '-' + ot[5:7] + '-' + ot[8:10] + 'T'
    timefits += ot[11:13] + ot[14:17] + ot[18:21] + '.000'
    pathSafeTimeString = timefits.replace(':', '-')
    time = Time(timefits, format='fits')
    y2000 = Time(2000, format='jyear')
    jdaysdelta = time.jd - y2000.jd
    jsecs = jdaysdelta*60*60*24
    return [jsecs, pathSafeTimeString]

fl_fits = fits.open(fitsfiles[11][0])
obstimeToJ2000(fl_fits[0].header['OBSTIME'])

[610099200.0000134, '2019-05-02T20-00-00.000']